In [59]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import random
import scipy.signal
from skimage.restoration import denoise_tv_bregman
from skimage import restoration
import skimage

In [62]:
random.seed(200)

tf.keras.backend.clear_session()
tf.config.optimizer.set_jit(False) # Start with XLA disabled.

def load_data():
  result = tfds.load('cifar10', batch_size = -1)
  (x_train, y_train) = result['train']['image'],result['train']['label']
  (x_test, y_test) = result['test']['image'],result['test']['label']

  x_train = x_train.numpy().astype('float32') / 256
  x_test = x_test.numpy().astype('float32') / 256

  # Convert class vectors to binary class matrices.
  y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
  y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)
  return ((x_train, y_train), (x_test, y_test))

(x_train, y_train), (x_test, y_test) = load_data()

def struct_noise(image, corr_scale, n):
    x = np.arange(-corr_scale, corr_scale)
    y = np.arange(-corr_scale, corr_scale)
    X, Y = np.meshgrid(x, y)
    dist = np.sqrt(X*X + Y*Y)
    filter_kernel = np.exp(-dist**2/(2*corr_scale))

    noise = np.random.randn(n, n)
    struct = []
    
    for i in range(0,3):
        struct.append(scipy.signal.fftconvolve(noise, filter_kernel, mode='same'))
        i += 1

    struct = np.transpose(struct, (1, 2, 0))
    img = image + struct

    return img

xs = (x * 0.25 for x in range(1, 4))
x_noisytrains = []
x_noisytests = []
x_tvtrains = []
x_tvtests = []

y_noisytrains = y_train
y_noisytests = y_test
y_tvtrains = y_train
y_tvtests = y_test

for i in xs:
    a,b = train_test_split(x_train, test_size=i, random_state=44)
    a0 = []
    atv = []
    for j in range(len(a)):
        a0.append(struct_noise(a[j], 50, 32))
    
    atv.append(skimage.restoration.denoise_tv_chambolle(a0))
    
    a0 = a0.append(b)
    atv = atv.append(b)

    x_noisytrains.append(a0)
    x_tvtrains.append(atv)
    
    e,f = train_test_split(x_test, test_size = i, random_state = 44)
    e0 = []
    etv = []
    for j in range(len(e)):
        e0.append(struct_noise(e[j], 50, 32))
        
    etv.append(skimage.restoration.denoise_tv_chambolle(e0))

    e0 = e0.append(f)
    etv = etv.append(f)
    
    x_noisytests.append(e0)
    x_tvtests.append(etv)

KeyboardInterrupt: 

In [ ]:
def generate_model():
  return tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(32, (3, 3)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Conv2D(64, (3, 3), padding='same'),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(64, (3, 3)),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(0.25),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Activation('softmax')
  ])

model = generate_model()


In [ ]:
def compile_model(model):
  opt = tf.keras.optimizers.adams(learning_rate=0.0001)
  model.compile(loss='categorical_crossentropy',
                optimizer=opt,
                metrics=['accuracy'])
  return model

model = compile_model(model)

def train_model(model, x_train, y_train, x_test, y_test, epochs=25):
  model.fit(x_train, y_train, batch_size=256, epochs=epochs, validation_data=(x_test, y_test), shuffle=True)

def warmup(model, x_train, y_train, x_test, y_test):
  # Warm up the JIT, we do not wish to measure the compilation time.
  initial_weights = model.get_weights()
  train_model(model, x_train, y_train, x_test, y_test, epochs=1)
  model.set_weights(initial_weights)

warmup(model, x_train, y_train, x_test, y_test)
%time train_model(model, x_train, y_train, x_test, y_test)

scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])
